This notebook can be used if you want to combine multiple runs together. Put each run in a CSV file and run the code.

In [ ]:
# oneshotCoT
import ollama
import csv
import pandas as pd
import numpy

runs = {}
methodRuns = 11
zeroshot = 'runs/zeroShotRuns/run'
oneshot = 'runs/oneShotRuns/run'
fewshot = 'runs/fewShotRuns/run'
zeroShotCoT = 'runs/zeroShotCoTRuns/run'
oneShotCoT = 'runs/oneShotCoTRuns/run'
fewShotCoT = 'runs/fewShotCoTRuns/run'

# this part reads each csv to a dataframe and ensures only the labels are copied
current = zeroshot
for n in range(1, 11):
    csvPath = current + str(n) + '.csv'
    res = pd.read_csv(csvPath, sep='{', header=None)
    res.columns = ["rows", "text"]
    res[["labels", "num"]] = res["text"].str.split("}", expand=True)
    runs["response" + str(n)] = res[["labels"]].copy()

display(runs["response2"])

In [ ]:
# From the response we create a dataframe in the same format as our annotated labels
# First we need a list of labels
labelsToAnnotate: str = "admiration;amusement;anger;annoyance;\"approval;caring;confusion;curiosity;desire;disappointment;disapproval;disgust;embarrassment;excitement;fear;gratitude;grief;joy;love;nervousness;optimism;pride;realization;relief;remorse;sadness;suprise;neutral"
labelsList = labelsToAnnotate.split(";")

# Then for each method run
for n in range (1, methodRuns):
   # Sometimes the LLM returns the labels with uppercases, these should be changed to lowercase
   runs["response" + str(n)]["labels"] = runs["response" + str(n)]["labels"].apply(lambda x: x.lower())
   # We go over each of these labels
   for label in labelsList:
      # And add a column with the correct name by d[][label]
      # Then for each row we either assign 1 if the label shows up in the text and 0 if the label does not show up in the text at all.
      # As the LLM sometimes gives emotions in the text followed by :0, those are filtered out. Other unneeded info is filtered likewise.
      runs["response" + str(n)][label] = runs["response" + str(n)].apply(lambda row: 0 if label + "\": 0," in row.labels else 0 if label + "\":0," in row.labels else 0 if label + "\": false," in row.labels else 1 if label in row.labels else  0, axis=1) 

# transform the result data to a numpy array to do evaluation on
frameResults = {}
numResults = {}

# For each method run
for n in range (1,methodRuns):
    # We drop the unneeded columns, which are the labels given by the LLM and a empty column it adds
    frameResults["response" + str(n)] = runs["response" + str(n)].drop(['labels'], axis=1)
    # then we turn the dataframe into a numpy array
    numResults["results" + str(n)] = pd.DataFrame.to_numpy(frameResults["response" + str(n)])

sumResults =  numResults["results1"]
for n in range (2, methodRuns):
    sumResults = numpy.add(sumResults, numResults["results" + str(n)])

# this printed array can be copied over to the resultEvaluation notebook
print(repr(sumResults))